# Identification of atypical WMH patterns

This notebook can be used to identify atypical WMH patterns, based on a N=3525 multicentre memory clinic cohort.

In [23]:
import numpy as np
import os
import pandas
import SimpleITK as sitk
from sklearn.neighbors import LocalOutlierFactor
import skops.io as sio
from tqdm.contrib import tenumerate

## Parameters

All relevant parameters for this script can be set in the box below, related to either the input, settings, or output.

### Input 

- **dir_lesion:** the directory that contains the binary lesion maps per subject
- **design_document:** spreadsheet containing two or more columns: *first column:* name of the lesion map for every subject (should exist in dir_lesion or a subfolder (see below)); *second column:* continuous output variable, e.g. a Z-score on a specific domain; *optionally* more columns with additional output variables
- **lesion_prevalence_filename:** file that contains the lesion prevalence map for this dataset. Can be downloaded from: https://doi.org/10.34894/FYL9ID 

In [5]:
dir_lesion = r""
design_document = r""

lesion_prevalence_filename = r"metavcimap_memory_clinic_n3525_mni_space.nii.gz"

In [12]:
df = pandas.read_excel(design_document, header=None)

# Initialize lesion matrix
nii = sitk.ReadImage(os.path.join(dir_lesion, df[0][0]))

raw_lesion_matrix = np.zeros((len(df.index), sitk.GetArrayViewFromImage(nii).size), np.int8)
for i, lesion_filename in tenumerate(df[0]):
    nii = sitk.ReadImage(os.path.join(dir_lesion, lesion_filename))
    raw_lesion_matrix[i] = sitk.GetArrayViewFromImage(nii).ravel()

  0%|          | 0/3525 [00:00<?, ?it/s]

In [14]:
lesion_prevalence = sitk.GetArrayFromImage(sitk.ReadImage(lesion_prevalence_filename)).ravel()
lesion_prevalence_mask = lesion_prevalence > 0

In [15]:
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.001)
y_pred = clf.fit_predict(raw_lesion_matrix[:,lesion_prevalence_mask])

In [16]:
# The LOF score is saved in negative_outlier_factor_, where lower (more negative) scores indicate outliers.
# These results are used in our paper.
df['negative_outlier_factor'] = clf.negative_outlier_factor_

## Novelty detection

LOF can be trained as a novelty detection method. According to the documentation, this will give slightly different results. Therefore we train this separately. 

The resulting novelty detection method is published together with this notebook on: https://doi.org/10.34894/FYL9ID . Download it there, so it can be applied to new unseen data; and used by other researchers.

In [18]:
novelty_clf = LocalOutlierFactor(n_neighbors=20, contamination=0.001, novelty=True)
novelty_clf.fit(raw_lesion_matrix[:,lesion_prevalence_mask])

LocalOutlierFactor(contamination=0.001, novelty=True)

In [25]:
# Save the model
sio.dump(novelty_clf, "wmh_atypical_patterns_model.sav")

## Using the trained novelty detection

From this point in the code, you can use the trained novelty detection method on your own data. The WMH segmentation map must be in MNI space.

In [26]:
novelty_clf = sio.load("wmh_atypical_patterns_model.sav", trusted=True)

In [ ]:
unseen_nii = sitk.ReadImage("filename.nii")
unseen_matrix = sitk.GetArrayViewFromImage(unseen_nii).ravel()

score = novelty_clf.score_samples([unseen_matrix[lesion_prevalence_mask]])

In [ ]:
# The following data member holds all LOF scores from the training data. You can use this to compare your subject against

novelty_clf.negative_outlier_factor_

print("Your patient has a LOF score of: %.3f" % score[0])
print("When comparing your patient to our N=3525 memory clinic patients:")
print("  %.1f" % float(np.sum(novelty_clf.negative_outlier_factor_  < score[0]) / 3525 * 100), " percent are more atypical (have a lower score)")
print("  %.1f" % float(np.sum(novelty_clf.negative_outlier_factor_ >= score[0]) / 3525 * 100), " percent are more typical (have a higher score)")